In [5]:
import re

engsects = {}
grcsects = {}

def secttext(fname,tgwork):
    
    curbook = ''
    cursect = ''
    curcit = ''
    innote = 0
    refwords = {}
    
    f = open(fname)
    
    
    for l in f:
        l = re.sub('\s+$','',l)
        if(re.search('<note type="footnote"',l)):
            innote = 1
        if(re.search('<pb',l)):
            innote = 0
            continue
            
        m = re.search('subtype="book" n="([0-9]+)"',l)
        if(m):
            curbook = m[1] + '.'
            
        m = re.search('subtype="section" n="([0-9]+)"',l)
        if(m):
            cursect= m[1]
            #print('cursect',cursect)
            curcit = tgwork + ':' + curbook + cursect
            if(curcit in refwords):
                print('duplicate!',curcit)
            

        #print('cur','['+cursect+']',l)
        if(innote or not cursect):
            continue
        rawwords = re.sub('<sic>[^<]+</sic>','',l)
        rawwords = re.sub('<bibl[^>]*>[^<]+</bibl>',' ',rawwords)
        rawwords = re.sub('<note[^>]+>.+</note>',' ',rawwords)
        rawwords = re.sub('<[^>]+>',' ',rawwords)
        rawwords = re.sub('^\s+','',rawwords)
        rawwords = re.sub('\s+',' ',rawwords)
        if(curcit in refwords):
            refwords[curcit] = refwords[curcit] + ' ' + rawwords
        else:
            refwords[curcit] = rawwords
        #print(refwords[curcit])
            
        #print(curcit,l)
        
    
    f.close()
    #print(refwords)
    return(refwords)
    
def dobiling(workn):
    global engsects
    global grcsects
    engfname = 'yonge/corr/corr.' + workn + '.yonge_eng1.xml'
    grcfname = 'cohn-corr/corr-' + workn + '.opp-grc1.xml'
    
    cursects = secttext(grcfname,workn)
    for foo in cursects:
        grcsects[foo] = cursects[foo]
    
    cursects = secttext(engfname,workn)
    for foo in cursects:
        engsects[foo] = cursects[foo]
        
        
    

for i in range(1,32):
    curworkn = str(i)
    if(i<10):
        curworkn = '0' + curworkn
    curwork = 'tlg0018.tlg0'+curworkn
    print('doing',curwork)
    dobiling(curwork)
    
toteng = 0
for foo in engsects:
    toteng = toteng + len(engsects[foo].split())
    if(not foo in grcsects):
        print('grc missing',foo)

print('eng',len(engsects),toteng,toteng/len(engsects))
    
totgrc = 0

for foo in grcsects:
    totgrc = totgrc + len(grcsects[foo].split())
    if(not foo in engsects):
        print('grc missing',foo)

print('grc',len(grcsects),totgrc,totgrc/len(grcsects))

    
    
    

doing tlg0018.tlg001
doing tlg0018.tlg002
doing tlg0018.tlg003
doing tlg0018.tlg004
doing tlg0018.tlg005
doing tlg0018.tlg006
doing tlg0018.tlg007
doing tlg0018.tlg008
doing tlg0018.tlg009
doing tlg0018.tlg010
doing tlg0018.tlg011
doing tlg0018.tlg012
doing tlg0018.tlg013
doing tlg0018.tlg014
doing tlg0018.tlg015
doing tlg0018.tlg016
doing tlg0018.tlg017
doing tlg0018.tlg018
doing tlg0018.tlg019
doing tlg0018.tlg020
doing tlg0018.tlg021
doing tlg0018.tlg022
doing tlg0018.tlg023
doing tlg0018.tlg024
doing tlg0018.tlg025
doing tlg0018.tlg026
doing tlg0018.tlg027
doing tlg0018.tlg028
doing tlg0018.tlg029
doing tlg0018.tlg030
doing tlg0018.tlg031
eng 7843 727842 92.80147902588295
grc missing tlg0018.tlg024:1.177
grc missing tlg0018.tlg024:1.178
grc missing tlg0018.tlg024:1.179
grc missing tlg0018.tlg024:1.180
grc missing tlg0018.tlg024:1.181
grc missing tlg0018.tlg024:1.182
grc missing tlg0018.tlg024:1.183
grc missing tlg0018.tlg024:1.184
grc missing tlg0018.tlg024:1.185
grc missing tlg001

In [100]:

def compgrceng(curcit):
    print(len(grcsects[curcit].split()),grcsects[curcit])
    print(len(engsects[curcit].split()),engsects[curcit])
    
    
compgrceng('tlg0018.tlg002:1.20')

51  ἵνα δὲ μὴ καθ’ ὡρισμένας χρόνων περιόδους ὑπολάβῃς τὸ θεῖόν τι ποιεῖν, ἀλλ’ εἰδῇς ἄδηλα καὶ ἀτέκμαρτα καὶ ἀκατάληπτα τῷ θνητῷ γένει τὰ δημιουργούμενα, ἐπιφέρει τὸ  „ὅτε ἐγένετο“, τὸ πότε κατὰ περιγραφὴν οὐ διορίζων· ἀπεριγράφως γὰρ γίνεται τὰ γινόμενα ὑπὸ τοῦ αἰτίου. ἀνῄρηται τοίνυν τὸ ἐν ἓξ ἡμέραις γεγενῆσθαι τὸ πᾶν.   
89  But, lest you should imagine that the Deity does anything   according to definite periods of time, while you should rather think that everything done by him is inscrutable in its nature, uncertain, unknown to, and incomprehensible by the race of mortal men. Moses adds the words, "when they were created," not defining the time when by any exact limitation, for what has been made by the Author of all things has no limitation. And in this way the idea is excluded, that the universe was created in six days.   


In [90]:
len(grcsects)

7901

In [120]:
from bs4 import BeautifulSoup, Tag
import re

philo = open('cohn-corr/corr-tlg0018.tlg001.opp-grc1.xml', encoding='utf-8', errors='ignore')

philo_soup = BeautifulSoup(philo,'lxml')
notes= philo_soup.find_all("note")
for note in notes:
    note.decompose()
 #removing marked mistakes from Prof Crane's corrected file (<sic> tag)
sic= philo_soup.find_all('sic')
for s in sic:
    s.decompose()
#removing page break tags
pages = philo_soup.find_all('pb')
for p in pages:
    p.decompose()
#removing line break tags
breaks = philo_soup.find_all('lb')
for b in breaks:
    b.decompose()
    

bibls = philo_soup.find_all('bibl')
for foo in bibls:
    print('\n',foo)


In [110]:
for index, divtag in enumerate(philo_soup.find_all('div', subtype= "book")):
    divtag['id'] = f'Book{index+3}'
    chapters= philo_soup.find_all('div', subtype= "chapter")
    for c_index, ctag in enumerate(chapters):
        ctag['n']= f'Book{index+3}Ch{c_index+1}'
        
        
        
#creating section tags
sections= philo_soup.find_all('p')
for sec_index, sec_tag in enumerate(sections):
      sec_tag['id']= f"Section{sec_index+1}"

In [111]:
philo_soup

<?xml version="1.0" encoding="UTF-8"?><?xml-model href="https://epidoc.stoa.org/schema/latest/tei-epidoc.rng" schematypens="http://relaxng.org/ns/structure/1.0"?><html><body><tei xmlns="http://www.tei-c.org/ns/1.0">
<teiheader xml:lang="eng">
<filedesc>
<titlestmt><title xml:lang="lat">De opificio mundi</title>
<author xml:lang="lat">Philo Judaeus</author>
<editor>Leopold Cohn</editor>
<funder>European Social Fund Saxony</funder>
<principal>Gregory Crane</principal>
<respstmt>
<persname xml:id="DDD">Digital Divide Data</persname>
<resp>Corrected and encoded the text</resp>
</respstmt>
<respstmt>
<persname>Gregory Crane</persname>
<resp>Editor-in-Chief, Perseus Digital Library</resp>
</respstmt>
<respstmt>
<persname>Matt Munson</persname>
<resp>Project Manager (University of Leipzig)</resp>
</respstmt>
<respstmt>
<persname>Annette Geßner</persname>
<resp>Project Assistant (University of Leipzig)</resp>
</respstmt>
<respstmt>
<persname>Thibault Clérice</persname>
<resp>Lead Developer (Un

In [116]:
import stanza

nlp = stanza.Pipeline(lang='grc', processors='tokenize')

2023-05-22 09:30:31 INFO: Loading these models for language: grc (Ancient_Greek):
| Processor | Package |
-----------------------
| tokenize  | proiel  |

2023-05-22 09:30:31 INFO: Use device: cpu
2023-05-22 09:30:31 INFO: Loading: tokenize
2023-05-22 09:30:31 INFO: Done loading processors!


In [117]:
doc = nlp("ὡς γὰρ τῶν κολοσσιαίων μεγεθῶν τὰς ἐμφάσεις καὶ ἡ βραχυτάτη σφραγὶς τυπωθεῖσα δέχεται")

In [118]:
doc

[
  [
    {
      "id": 1,
      "text": "ὡς",
      "start_char": 0,
      "end_char": 2
    },
    {
      "id": 2,
      "text": "γὰρ",
      "start_char": 3,
      "end_char": 6
    },
    {
      "id": 3,
      "text": "τῶν",
      "start_char": 7,
      "end_char": 10
    },
    {
      "id": 4,
      "text": "κολοσσιαίων",
      "start_char": 11,
      "end_char": 22
    },
    {
      "id": 5,
      "text": "μεγεθῶν",
      "start_char": 23,
      "end_char": 30
    },
    {
      "id": 6,
      "text": "τὰς",
      "start_char": 31,
      "end_char": 34
    },
    {
      "id": 7,
      "text": "ἐμφάσεις",
      "start_char": 35,
      "end_char": 43
    },
    {
      "id": 8,
      "text": "καὶ",
      "start_char": 44,
      "end_char": 47
    },
    {
      "id": 9,
      "text": "ἡ",
      "start_char": 48,
      "end_char": 49
    },
    {
      "id": 10,
      "text": "βραχυτάτη",
      "start_char": 50,
      "end_char": 59
    },
    {
      "id": 11,
      "text": "σ